<a href="https://colab.research.google.com/github/Arnoldzyf/Apziva-Projects/blob/main/1_happy_customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# !pip install pandas
import pandas as pd

# Load Data

Y = target attribute (Y) with values indicating 0 (unhappy) and 1 (happy) customers  
X1 = my order was delivered on time  
X2 = contents of my order was as I expected  
X3 = I ordered everything I wanted to order  
X4 = I paid a good price for my order  
X5 = I am satisfied with my courier  
X6 = the app makes ordering easy for me  

Attributes X1 to X6 indicate the responses for each question and have values from 1 to 5 where the smaller number indicates less and the higher number indicates more towards the answer.

In [8]:
path = "/content/drive/MyDrive/Apziva/Project 1 - Happy Customers/Resources/ACME-HappinessSurvey2020.csv"
survey_df = pd.read_csv(f"{path}")
survey_df

,Y,X1,X2,X3,X4,X5,X6
0,0,3,3,3,4,2,4
1,0,3,2,3,5,4,3
2,1,5,3,3,3,3,5
3,0,5,4,3,3,3,5
4,0,5,4,3,3,3,5
...,...,...,...,...,...,...,...
121,1,5,2,3,4,4,3
122,1,5,2,3,4,2,5
123,1,5,3,3,4,4,5
124,0,4,3,3,4,4,5


Very small portion of data, be careful with overfitting

## statistics

In [ ]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Y       126 non-null    int64
 1   X1      126 non-null    int64
 2   X2      126 non-null    int64
 3   X3      126 non-null    int64
 4   X4      126 non-null    int64
 5   X5      126 non-null    int64
 6   X6      126 non-null    int64
dtypes: int64(7)
memory usage: 7.0 KB


No missing values

In [ ]:
survey_df.describe()

,Y,X1,X2,X3,X4,X5,X6
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000
mean,0.547619,4.333333,2.531746,3.309524,3.746032,3.650794,4.253968
std,0.499714,0.800000,1.114892,1.023440,0.875776,1.147641,0.809311
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,4.000000,2.000000,3.000000,3.000000,3.000000,4.000000
50%,1.000000,5.000000,3.000000,3.000000,4.000000,4.000000,4.000000
75%,1.000000,5.000000,3.000000,4.000000,4.000000,4.000000,5.000000
max,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


\+ distribution of each column -- may be later

In [ ]:
corr = survey_df.corr(numeric_only = True)
corr.style.background_gradient(cmap='coolwarm')

,Y,X1,X2,X3,X4,X5,X6
Y,1.000000,0.280160,-0.024274,0.150838,0.064415,0.224522,0.167669
X1,0.280160,1.000000,0.059797,0.283358,0.087541,0.432772,0.411873
X2,-0.024274,0.059797,1.000000,0.184129,0.114838,0.039996,-0.062205
X3,0.150838,0.283358,0.184129,1.000000,0.302618,0.358397,0.203750
X4,0.064415,0.087541,0.114838,0.302618,1.000000,0.293115,0.215888
X5,0.224522,0.432772,0.039996,0.358397,0.293115,1.000000,0.320195
X6,0.167669,0.411873,-0.062205,0.203750,0.215888,0.320195,1.000000


? *color*

?? data augmentation

# Task

**Task Description**:
1. Predict if a customer is happy or not based on the answers they give to questions asked
  - Reach 73% accuracy score or above

2. what is the minimal set of attributes/features that would preserve the most information about the problem while increasing predictability of the data we have


**Task Analysis**:
1. Classification Problem ([sklearn](https://scikit-learn.org/stable/))
  - classification model
  - regression model + threshold
  - neural net
2. Feature Importance
  - test best model on validation set   
  - `feature_importances_` in sklearn `RandomForestClassifier`
  - ?? Other ways
    - delete iteratively (permutation)
    - dimension reduction (PCA, ...)
    - feature selection (coefficient, ...)
3. only need a very **small** model

# Preprocess Data

## Split Data

train : validation: test = 0.7 : 0.15 : 0.15 --> 88 : 19: 19





In [ ]:
X = survey_df.drop(columns=['Y'])
y = survey_df['Y']

In [ ]:
## "un" means "havn't been standardized"
from sklearn.model_selection import train_test_split
X_train_un, X_test_un, y_train_un, y_test_un = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_val_un, X_test_un, y_val_un, y_test_un = train_test_split(X_test_un, y_test_un, test_size=0.5, random_state=42)

## Standardize Data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train_un)

In [ ]:
X_train = scaler.transform(X_train_un)  ## zero mean, unit variance
# print(X_train.mean(axis=0))
# print(X_train.std(axis=0))

In [ ]:
X_val = scaler.transform(X_val_un)
X_test = scaler.transform(X_test_un)

print(X_val.mean(axis=0))
print(X_val.std(axis=0))
print(X_test.mean(axis=0))
print(X_test.std(axis=0))

[-0.33508861  0.0623175  -0.24391407  0.21276411 -0.22738092  0.3306097 ]
[1.03995122 0.70481418 0.88926192 0.88120698 0.98087284 0.73146025]
[-0.26155296 -0.54184542 -0.35096863  0.09500787 -0.22738092  0.07199945]
[1.54249693 0.99241458 1.25532593 0.93094478 1.14045265 1.11607628]


Not close to 0 and 1 --> The performance of `StandardScaler()` is **NOT** good.    
Maybe it's casued by the small data size, i.e. the distribution of training data cannot well represent the distribution of all the data.

In [ ]:
## no need to change Y
y_train, y_val, y_test = y_train_un, y_val_un, y_test_un

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression


clf_LR = LogisticRegression(random_state=0).fit(X_train, y_train)
print(f"training accuracy: {clf_LR.score(X_train, y_train)}")

training accuracy: 0.6477272727272727


In [ ]:
print(f"validation accuracy: {clf_LR.score(X_val, y_val)}")

validation accuracy: 0.42105263157894735


# Linear SVC

Not sure about its math details ...

In [ ]:
from sklearn.svm import LinearSVC

clf_LSVC = LinearSVC(random_state=0, tol=1e-5).fit(X_train, y_train)
print(f"training accuracy: {clf_LSVC.score(X_train, y_train)}")

training accuracy: 0.6477272727272727


In [ ]:
print(f"validation accuracy: {clf_LSVC.score(X_val, y_val)}")

validation accuracy: 0.42105263157894735


# K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf_KNN = KNeighborsClassifier(n_neighbors=3, leaf_size=4).fit(X_train, y_train)
print(f"training accuracy: {clf_KNN.score(X_train, y_train)}")

print(f"validation accuracy: {clf_KNN.score(X_val, y_val)}")

training accuracy: 0.7954545454545454
validation accuracy: 0.5789473684210527


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier(random_state=0, max_depth=4).fit(X_train, y_train)
print(f"training accuracy: {clf_RF.score(X_train, y_train)}")

print(f"validation accuracy: {clf_RF.score(X_val, y_val)}")

training accuracy: 0.7840909090909091
validation accuracy: 0.47368421052631576


In [ ]:
print(f"test accuracy: {clf_RF.score(X_test, y_test)}")

test accuracy: 0.6842105263157895


# ??

- hyper param tuning
- more data